In [1]:
from sklearn import datasets

In [2]:
digits = datasets.load_digits()

In [3]:
#Try Primitives approach

In [13]:
import gym
import numpy as np
import time
import math
import random
import operator
from IPython.display import clear_output
from gym import wrappers
from deap import algorithms
from deap import base
from deap import creator
from deap import tools
from deap import gp

In [235]:
pset = gp.PrimitiveSet("main", 64)
def if_then_else(input, output1, output2):
    return output1 if input else output2
def protectedDiv(left, right):
    try:
        return left / right
    except ZeroDivisionError:
        return 1
pset.addPrimitive(if_then_else, 3)

def protectedLog(left):
    return math.log10(abs(left)+1e-7)
pset.addPrimitive(if_then_else, 3)

pset.addPrimitive(operator.neg, 1)
pset.addPrimitive(max, 2)
pset.addPrimitive(min, 2)
pset.addPrimitive(protectedDiv, 2)
pset.addPrimitive(operator.add, 2)
pset.addPrimitive(operator.mul, 2)
pset.addPrimitive(operator.mul, 2)

pset.addTerminal(1)
pset.addTerminal(0)


In [236]:
expr = gp.genFull(pset, min_=1, max_=3)
tree = gp.PrimitiveTree(expr)

In [237]:
import multiprocessing

In [241]:
def compare(data, value):
    if math.isnan(data): data=9999999999
    
    if value-np.log10(data)>0.3 and value-np.log10(data)<=0.7:
        return 1
    
    return 0

def evalSymbReg(individual, n_times=1000):
    func = toolbox.compile(expr=individual)

    rewards = [compare(func(*digits.data[i]),digits.target[i]) for i in range(n_times)]
    
    return float(np.mean(rewards)),

In [242]:
creator.create("FitnessMin", base.Fitness, weights=(1.0,))
creator.create("Individual", gp.PrimitiveTree, fitness=creator.FitnessMin,
               pset=pset)


In [243]:
toolbox = base.Toolbox()
toolbox.register("expr", gp.genHalfAndHalf, pset=pset, min_=1, max_=7)
toolbox.register("individual", tools.initIterate, creator.Individual, toolbox.expr)
# toolbox.register("individual_all", tools.initRepeat, container=list, func=toolbox.individual,  n=10)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)
toolbox.register("compile", gp.compile, pset=pset)

toolbox.register("evaluate", evalSymbReg)
toolbox.register("select", tools.selDoubleTournament, fitness_size=7, parsimony_size=1, fitness_first=True)
toolbox.register("mate", gp.cxOnePoint)
toolbox.register("expr_mut", gp.genFull, min_=0, max_=3)
toolbox.register("mutate", gp.mutUniform, expr=toolbox.expr_mut, pset=pset)

toolbox.decorate("mate", gp.staticLimit(key=operator.attrgetter("height"), max_value=25))
toolbox.decorate("mutate", gp.staticLimit(key=operator.attrgetter("height"), max_value=25))

In [244]:
stats_fit = tools.Statistics(lambda ind: ind.fitness.values)
stats_size = tools.Statistics(len)
mstats = tools.MultiStatistics(fitness=stats_fit, size=stats_size)
mstats.register("avg", np.mean)
mstats.register("std", np.std)
mstats.register("min", np.min)
mstats.register("max", np.max)

In [ ]:
pool = multiprocessing.Pool(processes=4)
toolbox.register("map", pool.map)
pop = toolbox.population(n=300)
hof = tools.HallOfFame(1)
pop, log = algorithms.eaSimple(pop, toolbox, 0.5, 0.3, 400, stats=mstats,
                                   halloffame=hof, verbose=True)

/Users/oleksandrsavsunenko/anaconda3/envs/data/lib/python3.7/site-packages/ipykernel_launcher.py:4: RuntimeWarning: divide by zero encountered in log10
  after removing the cwd from sys.path.
/Users/oleksandrsavsunenko/anaconda3/envs/data/lib/python3.7/site-packages/ipykernel_launcher.py:4: RuntimeWarning: invalid value encountered in log10
  after removing the cwd from sys.path.
/Users/oleksandrsavsunenko/anaconda3/envs/data/lib/python3.7/site-packages/ipykernel_launcher.py:4: RuntimeWarning: divide by zero encountered in log10
  after removing the cwd from sys.path.
/Users/oleksandrsavsunenko/anaconda3/envs/data/lib/python3.7/site-packages/ipykernel_launcher.py:6: RuntimeWarning: invalid value encountered in double_scalars
  
/Users/oleksandrsavsunenko/anaconda3/envs/data/lib/python3.7/site-packages/ipykernel_launcher.py:6: RuntimeWarning: invalid value encountered in double_scalars
  
/Users/oleksandrsavsunenko/anaconda3/envs/data/lib/python3.7/site-packages/ipykernel_launcher.py:4:

   	      	                 fitness                 	              size             
   	      	-----------------------------------------	-------------------------------
gen	nevals	avg       	max  	min	std      	avg    	max	min	std    
0  	300   	0.00666333	0.056	0  	0.0107757	41.5533	538	2  	88.9307
1  	214   	0.0173667 	0.086	0  	0.0140603	10.8933	204	1  	19.6896
2  	184   	0.0291867 	0.091	0  	0.0194219	10.8133	181	1  	15.9074
3  	198   	0.0444167 	0.104	0  	0.0264195	14.2667	61 	1  	16.5304
4  	190   	0.0610033 	0.104	0  	0.0325607	16.55  	64 	1  	17.6845
5  	189   	0.07303   	0.106	0  	0.0333366	19.3133	74 	1  	17.9702
6  	216   	0.0819633 	0.108	0  	0.0313318	26.9733	82 	1  	21.9014
7  	196   	0.0899633 	0.108	0  	0.0279885	42.67  	119	1  	22.8925
8  	211   	0.0900233 	0.108	0  	0.0292155	38.4967	126	1  	26.5468
9  	203   	0.0884233 	0.112	0  	0.032093 	31.5933	105	1  	21.9885
10 	207   	0.0950333 	0.114	0  	0.0263489	37.87  	84 	1  	14.3462
11 	183   	0.100687  	0.125	0  	0.0216

In [ ]:
q =toolbox.compile(hof[0])

In [179]:
results = []
for i in range(1000,1300):
    r = q(*digits.data[i])
    for j in range(0,9):
        if (r / 10**j)>0.3 and (r / 10**j)<0.7:
            answer=j
            break
        answer=0
    if answer == digits.target[i]:
        results.append(1)
    else:
        results.append(0)

In [180]:
np.mean(results)

0.17